In [1]:
import numpy as np
import sys, os
import subprocess
import matplotlib.pyplot as plt
from shutil import copyfile
# adjust some settings for matplotlib
from matplotlib import rcParams
import pynoddy
# print rcParams
# determine path of repository to set paths corretly below
#repo_path = os.path.realpath('../..')

In [3]:
basepynoddyfile = pynoddy.__file__[:-11]+'experiment/__init__.py'
# Read in the file
with open(basepynoddyfile, 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('from . import util.sampling as Sample', 'from .util import sampling as Sample')

# Write the file out again
with open(basepynoddyfile, 'w') as file:
  file.write(filedata)
  
target = pynoddy.__file__[:-11]+'output.py'

source = 'output.py'

copyfile(source, target)

'C:\\Users\\ahinoamp\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\pynoddy\\output.py'

In [4]:
import pynoddy.history
import pynoddy.experiment
import pynoddy.events

In [7]:
folder = !pwd
folder = folder[0]
noddyEXE = folder+'/noddy.exe'
strV = 'chmod 777 '+noddyEXE
!eval "$strV"

'eval' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
modelfile = 'ThreeLayerTrenton.his'
#pynoddyEXEfile = '/content/DataAndCode/noddy_Mac.exe'
output_name = 'noddy_out'

H1 = pynoddy.history.NoddyHistory(modelfile)
H1.change_cube_size(250)
H1.events[3].properties['Dip'] = 40
H1.events[4].properties['Dip'] = 40
H1.events[5].properties['Dip'] = 40
H1.events[5].properties['X'] = 8000
H1.write_history(modelfile)
print('cube size is ' +str(H1.get_cube_size()))
print(subprocess.Popen([noddyEXE, modelfile, output_name, 'ALL'], 
                       shell=False, stderr=subprocess.PIPE, 
                       stdout=subprocess.PIPE).stdout.read())

cube size is 250.0


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [6]:
!ls

 apt.txt		        noddy_Mac
 arr.pvd		        noddy_out.dxf
'auto_rendering (1).ipynb'      noddy_out.g00
 auto_rendering.ipynb	        noddy_out.g01
 core.python.140.1578098860     noddy_out.g02
 core.python.168.1578098868     noddy_out.g12
 core.python.53.1578098844      noddy_out.grv
 DisplacementTransferZone.his   noddy_out.mag
 DTZ_3d.png		        noddy_out_play.vtr
 environment.yml	        noddy_out.vtr
 kkm.vtr		        output.py
 kk.pvtr		        Points.csv
 legacy.vtk		        postBuild
'Main (1)-Copy1.ipynb'	        ReallyNiceTrentonModel.his
'Main (1).ipynb'	        setup.py
 Main.ipynb		        ThreeLayerTrenton.his
 noddyBatchProgress.txt         WellNamesPaths.csv
 noddy.exe


In [13]:
import ezdxf
import pandas as pd
filename = 'noddy_out.dxf'
doc = ezdxf.readfile(filename)

def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf.layer)
    print("start point1: %s\n" % e.dxf.vtx0)
    print("end point2: %s\n" % e.dxf.vtx1)
    print("end point3: %s\n" % e.dxf.vtx2)
    print("end point4: %s\n" % e.dxf.vtx3)


msp = doc.modelspace()
faces = msp.query('3DFACE')
num3Dfaces=len(faces)
print(num3Dfaces)

# %%


i=0
x = np.zeros((num3Dfaces*3, 1))
y = np.zeros((num3Dfaces*3, 1))
z = np.zeros((num3Dfaces*3, 1))
name = []
for e in faces:
    x[i] = e.dxf.vtx0[0]
    y[i] = e.dxf.vtx0[1]
    z[i] = e.dxf.vtx0[2]
    name.append(e.dxf.layer)

    x[i+1] = e.dxf.vtx1[0]
    y[i+1] = e.dxf.vtx1[1]
    z[i+1] = e.dxf.vtx1[2]
    name.append(e.dxf.layer)

    x[i+2] = e.dxf.vtx2[0]
    y[i+2] = e.dxf.vtx2[1]
    z[i+2] = e.dxf.vtx2[2]
    name.append(e.dxf.layer)
    i=i+3
print(i)

# %%
#remove row duplicates        
AllPoints = pd.DataFrame({'x': x.reshape((-1,)), 'y':y.reshape((-1,)), 'z': z.reshape((-1,)), 'lith': name})
AllPoints = AllPoints.drop_duplicates(subset=['x', 'y', 'z'], keep='first')
AllPoints.to_csv('Points.csv', index=False)


463763
1391289


In [2]:
import ezdxf
import pandas as pd
filename = 'noddy_out.dxf'
doc = ezdxf.readfile(filename)

In [17]:
import numpy as np
import meshio

msp = doc.modelspace()
faces = msp.query('3DFACE')
num3Dfaces=len(faces)
print(num3Dfaces)
points = np.zeros((num3Dfaces*3, 3))

cells = np.zeros((num3Dfaces, 3),dtype ='int')
cell_data=[]

i=0
faceCounter=0
name = []
for e in faces:
    points[i, 0] = e.dxf.vtx0[0]
    points[i, 1] = e.dxf.vtx0[1]
    points[i, 2] = e.dxf.vtx0[2]

    points[i+1, 0] = e.dxf.vtx1[0]
    points[i+1, 1] = e.dxf.vtx1[1]
    points[i+1, 2] = e.dxf.vtx1[2]

    points[i+2, 0] = e.dxf.vtx2[0]
    points[i+2, 1] = e.dxf.vtx2[1]
    points[i+2, 2] = e.dxf.vtx2[2]

    cells[faceCounter,:]= [i, i+1, i+2]
    cell_data.append(e.dxf.layer)
    i=i+3
    faceCounter=faceCounter+1
print(i)

cell_data = pd.Series(cell_data)

CatCodes = np.zeros((len(cell_data),))
filterB = (cell_data.str.contains('B')) 
filterS = (cell_data.str.contains('S')) 

CatCodes[filterB]= cell_data.loc[filterB].str[:-16].astype('category').cat.codes
CatCodes[filterS]= -1*cell_data.loc[filterS].str[:-12].astype('category').cat.codes

#CatCodes[filterB]= 1
#CatCodes[filterS]= 2

# %%
UniqueCodes = np.unique(CatCodes)
nCodes = len(UniqueCodes)

meshio.write_points_cells(
    "fooColor.vtk",
    points,
    cells={'triangle':cells},
    cell_data= {'triangle': {'cat':CatCodes}}   
    )

463763
1391289


In [27]:
import numpy as np
import meshio

msp = doc.modelspace()
faces = msp.query('3DFACE')
num3Dfaces=len(faces)
print(num3Dfaces)
points = np.zeros((num3Dfaces*3, 3))

cells = np.zeros((num3Dfaces, 3),dtype ='int')
cell_data=[]

i=0
faceCounter=0
name = []
for e in faces:
    points[i, 0] = e.dxf.vtx0[0]
    points[i, 1] = e.dxf.vtx0[1]
    points[i, 2] = e.dxf.vtx0[2]

    points[i+1, 0] = e.dxf.vtx1[0]
    points[i+1, 1] = e.dxf.vtx1[1]
    points[i+1, 2] = e.dxf.vtx1[2]

    points[i+2, 0] = e.dxf.vtx2[0]
    points[i+2, 1] = e.dxf.vtx2[1]
    points[i+2, 2] = e.dxf.vtx2[2]

    cells[faceCounter,:]= [i, i+1, i+2]
    cell_data.append(e.dxf.layer)
    i=i+3
    faceCounter=faceCounter+1
print(i)

cell_data = pd.Series(cell_data)

CatCodes = np.zeros((len(cell_data),))
filterB = (cell_data.str.contains('B')) 
filterS = (cell_data.str.contains('S')) 

CatCodes[filterB]= cell_data.loc[filterB].str[:-16].astype('category').cat.codes
CatCodes[filterS]= -1*cell_data.loc[filterS].str[:-12].astype('category').cat.codes

#CatCodes[filterB]= 1
#CatCodes[filterS]= 2

# %%
UniqueCodes = np.unique(CatCodes)
nCodes = len(UniqueCodes)

UniqueCodes = np.unique(CatCodes)
nCodes = len(UniqueCodes)
for i in range(nCodes):
    filterPoints = CatCodes==UniqueCodes[i]
    nCells = np.sum(filterPoints)
    Cells_i = np.zeros((nCells, 3),dtype ='int')
    cntr = 0
    for j in range(nCells):
        Cells_i[j]=[cntr, cntr+1, cntr+2]
        cntr=cntr+3
        
    meshio.write_points_cells(
        "fooTry"+str(i)+".vtk",
        points[np.repeat(filterPoints,3), :],
        cells={'triangle':Cells_i}
        )

463763
1391289


In [21]:
for i

array([ 9, 10, 11])

In [ ]:
i=0
faceCounter=0
name = []
for e in faces:
    points[i, 0] = e.dxf.vtx0[0]
    points[i, 1] = e.dxf.vtx0[1]
    points[i, 2] = e.dxf.vtx0[2]

    points[i+1, 0] = e.dxf.vtx1[0]
    points[i+1, 1] = e.dxf.vtx1[1]
    points[i+1, 2] = e.dxf.vtx1[2]

    points[i+2, 0] = e.dxf.vtx2[0]
    points[i+2, 1] = e.dxf.vtx2[1]
    points[i+2, 2] = e.dxf.vtx2[2]

    cells[faceCounter,:]= [i, i+1, i+2]
    cell_data.append(e.dxf.layer)
    i=i+3
    faceCounter=faceCounter+1
print(i)

cell_data = pd.Series(cell_data)

CatCodes = np.zeros((len(cell_data),))
filterB = (cell_data.str.contains('B')) 
filterS = (cell_data.str.contains('S')) 

CatCodes[filterB]= cell_data.loc[filterB].str[:-16].astype('category').cat.codes
CatCodes[filterS]= -1*cell_data.loc[filterS].str[:-12].astype('category').cat.codes

CatCodes[filterB]= 1
CatCodes[filterS]= 2

# %%
UniqueCodes = np.unique(CatCodes)
nCodes = len(UniqueCodes)
for i in range(nCodes):
    filterPoints = CatCodes==UniqueCodes[i]
    meshio.write_points_cells(
        "fooTry"+str(i)+".vtk",
        points[np.repeat(filterPoints,3), :],
        cells={'triangle':cells[filterPoints, :]}
        )

In [9]:
UniqueCodes = np.unique(CatCodes)
nCodes = len(UniqueCodes)
for i in range(nCodes):
    filterPoints = CatCodes==UniqueCodes[i]
    meshio.write_points_cells(
        "foo"+str(i)+".vtk",
        points[np.repeat(filterPoints,3), :],
        cells={'triangle':cells[filterPoints, :]}
        )

463763

In [8]:
len(np.repeat(filterPoints,3))

1391289

In [49]:
cell_data = pd.Series(cell_data)

CatCodes = np.zeros((len(cell_data),))
filterB = (cell_data.str.contains('B')) 
filterS = (cell_data.str.contains('S')) 

CatCodes[filterB]= cell_data[filterB].str[:-16].astype('category').cat.codes
CatCodes[filterS]= -1*cell_data[filterS].str[:-12].astype('category').cat.codes


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [59]:
cell_data[0:1]

lith    lith    lith    lith    0         S_00_02_001_...
dtype: object

In [8]:
output_name = 'noddy_out'
N1 = pynoddy.output.NoddyOutput(output_name)

In [9]:
print("The model has an extent of %.0f m in x-direction, with %d cells of width %.0f m" %
      (N1.extent_x, N1.nx, N1.delx))

The model has an extent of 14000 m in x-direction, with 56 cells of width 250 m


In [10]:
N1.export_to_vtk()

In [11]:
def polyfit2d(x, y, z, order=3):
    ncols = (order + 1)**2
    G = np.zeros((x.size, ncols))
    ij = itertools.product(range(order+1), range(order+1))
#    ij = [tup for tup in ij if sum(tup) <= order]
    for k, (i,j) in enumerate(ij):
        G[:,k] = x**i * y**j
    m, _, _, _ = np.linalg.lstsq(G, z)
    return m

def polyval2d(x, y, m):
    order = int(np.sqrt(len(m))) - 1
    ij = itertools.product(range(order+1), range(order+1))
    z = np.zeros_like(x)
    for a, (i,j) in zip(m, ij):
        z += a * x**i * y**j
    return z

In [27]:
import pandas as pd
from vtkplotter import *
import numpy as np
import matplotlib.pylab as pl
import itertools
import k3d
from scipy.interpolate import griddata
import vtk

def GetCat(strCat):
    CatCodes = np.zeros((len(strCat),))
    filterB = (strCat['lith'].str.contains('B')) 
    filterS = (strCat['lith'].str.contains('S')) 
    
    CatCodes[filterB]= strCat.loc[filterB, 'lith'].str[:-16].astype('category').cat.codes
    CatCodes[filterS]= -1*strCat.loc[filterS, 'lith'].str[:-12].astype('category').cat.codes

#    CatCodes[filterB]= 0
#    CatCodes[filterS]= 1

    return CatCodes


embedWindow('k3d') #uncomment to pop the usual vtk rendering window, press Esc to close it.

positions = np.random.randn(5000,3)

data = pd.read_csv('Points.csv')
data['Cat'] = GetCat(data)

positions2 = data[['x', 'y','z']].values
scalars = data['Cat']

plot = Plotter()
#plot = k3d.plot()

plot.xtitle = '\sum_i x_i'


listSurfaces = np.unique(data['Cat'])
nSurfaces = len(listSurfaces)
colors = pl.cm.jet(np.linspace(0,1,nSurfaces))

for i in range(nSurfaces):
    surfaceId = listSurfaces[i]
    filterSurface = data['Cat']==surfaceId
    print(np.shape(positions2))

    points = Points(positions2[filterSurface]).color(colors[i][0:3])
    # color vertices based on their scalar value with any matplotlib color map
#    points.pointColor(colors[i])
#    plot += points

    reco = recoSurface(points, bins=128).legend("surf reco")
#    plt_surface = k3d.surface(zz, xmin=x.min(), xmax=x.max(), ymin=y.min(), ymax=y.max())

    plot += reco

filename = 'kkm.vtr'
e=load(filename)
#e = loadRectilinearGrid(filename)
#reader = vtk.vtkRectilinearGridReader()
#reader.SetFileName(filename)
#reader.Update()
#routput = reader.GetOutput()
#e = Actor(routput)

value = e.info.keys()
scals = e.scalars('geology')
e.pointColors(scals, cmap='Set3')

plot += e

#plot.show()
#plot.display()

(23275, 3)
Recalculating normals for 4931 Points, sample size= 98
Calculating mesh from points with R = 542.9252451182724
(23275, 3)
Recalculating normals for 4517 Points, sample size= 90
Calculating mesh from points with R = 555.1824375865258
(23275, 3)
Recalculating normals for 2422 Points, sample size= 48
Calculating mesh from points with R = 348.9759449929405
(23275, 3)
Recalculating normals for 2510 Points, sample size= 50
Calculating mesh from points with R = 348.9759449929405
(23275, 3)
Recalculating normals for 2607 Points, sample size= 52
Calculating mesh from points with R = 351.4607599877364
(23275, 3)
Recalculating normals for 2370 Points, sample size= 47
Calculating mesh from points with R = 348.9759449929405
(23275, 3)
Recalculating normals for 1505 Points, sample size= 30
Calculating mesh from points with R = 301.82733317800216
(23275, 3)
Recalculating normals for 2413 Points, sample size= 48
Calculating mesh from points with R = 343.9178056904336
❌ pointColors Error: nr

In [ ]:
plot.close()

In [ ]:
np.unique(data['Cat'])

In [ ]:
colors[i]

In [ ]:

import numpy as np
import k3d

plot = k3d.plot()

Nx, Ny = 40, 80
xmin, xmax, ymin, ymax = -3, 3, 0, 3

x = np.linspace(xmin, xmax, Nx, dtype=np.float32)
y = np.linspace(ymin, ymax, Ny, dtype=np.float32)
x, y = np.meshgrid(x, y)

heights = np.sin(x**2 + y**2)

plt_surface = k3d.surface(heights, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
plot += plt_surface

plot.display()

In [1]:
import pandas as pd
from vtkplotter import *

filename = 'noddy_out.vtr'
e=load(filename)
scals = e.scalars('geology')
e.pointColors(scals, cmap='Set3')

plot += e

plot.show()

❌ pointColors Error: nr. of scalars != nr. of points 7712 7714


NameError: name 'plot' is not defined

In [119]:
from vtkplotter import *

embedWindow('k3d') #uncomment to pop the usual vtk rendering window, press Esc to close it.

N1 = pynoddy.output.NoddyOutput(output_name)

x = np.arange(0, N1.extent_x, N1.delx, dtype='float64')
y = np.arange(0, N1.extent_y, N1.dely, dtype='float64')
z = np.arange(0, N1.extent_z, N1.delz, dtype='float64')
color = N1.block

#PointsPlot = np.concatenate((x.reshape(-1,1), y.reshape(-1,1), z.reshape(-1,1)), axis=1)
AAA= np.meshgrid(x,y,z)

x=AAA[0].reshape(-1,1)
y=AAA[1].reshape(-1,1)
z=AAA[2].reshape(-1,1)

Coord = np.concatenate((x,y,z), axis=1)

plot = Plotter()
print(np.shape(Coord))
points = Points(Coord).pointColors(np.transpose(color, axes =(1, 0, 2)).reshape(-1,1), cmap='Set3')
#plot += points

print(np.shape(np.transpose(color, axes =(2, 1, 0))))
             
#vol = Volume(np.transpose(color, axes =(2, 1, 0)), c='prism', spacing=[N1.delx, N1.dely,N1.delz])

xf = np.linspace(x.min(), x.max(), 40, dtype=np.float32)
yf = np.linspace(y.min(), y.max(), 40, dtype=np.float32)
xx, yy = np.meshgrid(xf, yf)

##    zfun_smooth_rbf = interp.Rbf(x, y, z, function='cubic', smooth=0)  # default smooth=0 for interpolation
##    zz = zfun_smooth_rbf(xx, yy)  # not really a function, but a callable class instance

pointsE=np.concatenate((x.reshape(-1,1), y.reshape(-1,1), z.reshape(-1,1)), axis=1)
print(np.shape(pointsE))
print(np.shape(color.reshape(-1,)))
print(np.shape(xx))
print(np.shape(yy))

newc = griddata(pointsE, color.reshape(-1,), (xx, yy, zz), method='nearest')



vol = Volume(newc, c='prism')

lego = vol.legosurface(-1, 4)


plot += lego

print('scalar min, max =', np.min(color), np.max(color))

plot.show()

(40320, 3)
(20, 36, 56)
(40320, 3)
(40320,)
(40, 40)
(40, 40)


TypeError: SetDimensions argument 1: expected a sequence of 3 values, got 2 values

In [19]:
PointsPlot

[array([[[    0.,     0.,     0., ...,     0.,     0.,     0.],
         [  250.,   250.,   250., ...,   250.,   250.,   250.],
         [  500.,   500.,   500., ...,   500.,   500.,   500.],
         ...,
         [13500., 13500., 13500., ..., 13500., 13500., 13500.],
         [13750., 13750., 13750., ..., 13750., 13750., 13750.],
         [14000., 14000., 14000., ..., 14000., 14000., 14000.]],
 
        [[    0.,     0.,     0., ...,     0.,     0.,     0.],
         [  250.,   250.,   250., ...,   250.,   250.,   250.],
         [  500.,   500.,   500., ...,   500.,   500.,   500.],
         ...,
         [13500., 13500., 13500., ..., 13500., 13500., 13500.],
         [13750., 13750., 13750., ..., 13750., 13750., 13750.],
         [14000., 14000., 14000., ..., 14000., 14000., 14000.]],
 
        [[    0.,     0.,     0., ...,     0.,     0.,     0.],
         [  250.,   250.,   250., ...,   250.,   250.,   250.],
         [  500.,   500.,   500., ...,   500.,   500.,   500.],
      

In [ ]:
voxels

In [115]:
#def voxels(voxels, color_map=nice_colors, wireframe=False, outlines=True, outlines_color=0, opacity=1.0, name=None,
#           compression_level=0, bounds=None,
#           **kwargs):

import numpy as np
import k3d
import scipy.interpolate as interp

embedWindow('itk') #uncomment to pop the usual vtk rendering window, press Esc to close it.


plot = k3d.plot()


voxels = k3d.voxels(np.transpose(color, axes =(2, 1, 0)), bounds=[0, 12000, 0, 7000, 0, 5000])
plot += voxels

listSurfaces = np.unique(data['Cat'])
nSurfaces = len(listSurfaces)
colors = pl.cm.jet(np.linspace(0,1,nSurfaces))

for i in range(nSurfaces):
    surfaceId = listSurfaces[i]
    filterSurface = data['Cat']==surfaceId
    print(np.shape(positions2))

    points = Points(positions2[filterSurface]).color(colors[i][0:3])
    # color vertices based on their scalar value with any matplotlib color map
#    points.pointColor(colors[i])
#    plot += points
    x = positions2[filterSurface, 0]
    y = positions2[filterSurface, 1]
    z = positions2[filterSurface, 2]

    xf = np.linspace(x.min(), x.max(), 40, dtype=np.float32)
    yf = np.linspace(y.min(), y.max(), 40, dtype=np.float32)
    xx, yy = np.meshgrid(xf, yf)
    
##    zfun_smooth_rbf = interp.Rbf(x, y, z, function='cubic', smooth=0)  # default smooth=0 for interpolation
##    zz = zfun_smooth_rbf(xx, yy)  # not really a function, but a callable class instance

    zz = griddata(np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1), z, (xx, yy), method='nearest')

    print(np.shape(np.concatenate((x.reshape(-1,1), y.reshape(-1,1)), axis=1)))
    print(np.shape(z))
    print(np.shape(xx))
    print(np.shape(yy))

    #    reco = recoSurface(points, bins=128).legend("surf reco")
    plt_surface = k3d.surface(zz, xmin=x.min(), xmax=x.max(), ymin=y.min(), ymax=y.max())

    plot += plt_surface


plot.display()

(23275, 3)
(4931, 2)
(4931,)
(40, 40)
(40, 40)
(23275, 3)
(4517, 2)
(4517,)
(40, 40)
(40, 40)
(23275, 3)
(2422, 2)
(2422,)
(40, 40)
(40, 40)
(23275, 3)
(2510, 2)
(2510,)
(40, 40)
(40, 40)
(23275, 3)
(2607, 2)
(2607,)
(40, 40)
(40, 40)
(23275, 3)
(2370, 2)
(2370,)
(40, 40)
(40, 40)
(23275, 3)
(1505, 2)
(1505,)
(40, 40)
(40, 40)
(23275, 3)
(2413, 2)
(2413,)
(40, 40)
(40, 40)


Output()

In [98]:
N1.delx*56

14000.0

In [103]:
from __future__ import print_function, division
import numpy as np
from vtkplotter import *
embedWindow('itkwidgets') # backends are: itkwidgets, k3d or False

X, Y, Z = np.mgrid[:30, :30, :30]
# scaled distance from the center at (15, 15, 15)
scalar_field = ((X-15)**2 + (Y-15)**2 + (Z-15)**2)/225/3
print('scalar min, max =', np.min(scalar_field), np.max(scalar_field))

vol = Volume(scalar_field)
lego = vol.legosurface(vmin=0, vmax=1.1)

#show(vol.c('magma'))
show(lego)

scalar min, max = 0.0 1.0


Viewer(cmap='jet', geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_point…

In [1]:
import meshio
import numpy as np

points = np.array([
    [0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    ])
cells = {
    "triangle": np.array([
        [0, 1, 2]
        ])
    }
meshio.write_points_cells(
    "foo.vtk",
    points,
    cells,
    # Optionally provide extra data on points, cells, etc.
    # point_data=point_data,
    # cell_data=cell_data,
    # field_data=field_data
    )

In [26]:

points = np.array([
    [0.0, 0.0, 0.0],
[1.0, 0.0, 0.0],
[2.0, 0.0, 0.0],
[0.0, 1.0, 0.0],
[1.0, 1.0, 0.0],
[2.0, 1.0, 2]])

# Define connectivity or vertices that belongs to each element
cells = {
    "triangle": np.array([
        [0, 1, 3]
        , [1, 4, 3]
        ]),
    "quad": np.array([
        [1, 2, 5, 4]
        ])
    }

meshio.write_points_cells(
    "foo.vtk",
    points,
    cells,
    # Optionally provide extra data on points, cells, etc.
    # point_data=point_data,
    # cell_data=cell_data,
    # field_data=field_data
    )

In [27]:
cells

{'triangle': array([[0, 1, 3],
        [1, 4, 3]]), 'quad': array([[1, 2, 5, 4]])}

In [ ]:
import pandas as pd
from vtkplotter import *
import numpy as np
#import matplotlib.pylab as pl
import itertools
import k3d
from scipy.interpolate import griddata
import vtk

plot = Plotter()

embedWindow('k3d')

filename = 'foo28.vtk'
e=load(filename)

plot += e

plot.show()

In [36]:
import pandas as pd
from vtkplotter import *
import numpy as np
#import matplotlib.pylab as pl
import itertools
import k3d
from scipy.interpolate import griddata
import vtk
import matplotlib
import matplotlib.pylab as pl

plot = Plotter()

embedWindow('k3d')

nSurfaces = 29
colors = pl.cm.jet(np.linspace(0,1,nSurfaces))

for i in range(nSurfaces):
    print(i)
    filename = 'fooTry'+str(i)+'.vtk'
    e=load(filename).c(colors[i, 0:3])

    plot += e

plot.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[6146.346336284672,…

In [31]:
!pip install matplotlib

In [32]:
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [18]:
import pandas as pd
from vtkplotter import *
import numpy as np
#import matplotlib.pylab as pl
import itertools
import k3d
from scipy.interpolate import griddata
import vtk

plot = Plotter()

embedWindow('k3d')
i=1
print(i)
filename = 'fooColor.vtk'
e=load(filename)

plot += e

plot.show()

1


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[6146.346336284672,…